# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 35

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# import player data 
filepath = '../data/fbref/player_stats_week' + str(latest_gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)
playerStats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,28,1990,28,28,2424,3,2,1,1,0,0,0.11,0.07,0.19,0.07,0.15,2.3,1.5,3.2,0.09,0.12,0.20,0.06,0.18,Matches
2,Max Aarons\Max-Aarons,eng ENG,DF,Norwich City,19,2000,33,33,2970,0,1,0,0,7,0,0.00,0.03,0.03,0.00,0.03,0.5,0.5,3.7,0.02,0.11,0.13,0.02,0.13,Matches
3,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,21,1997,31,25,2165,14,3,0,0,2,0,0.58,0.12,0.71,0.58,0.71,14.5,14.5,2.5,0.60,0.10,0.71,0.60,0.71,Matches
4,Che Adams\Che-Adams,eng ENG,FW,Southampton,23,1996,27,11,957,1,2,0,0,0,0,0.09,0.19,0.28,0.09,0.28,2.8,2.8,1.0,0.26,0.09,0.35,0.26,0.35,Matches
5,Adrián\Adrian,es ESP,GK,Liverpool,32,1987,11,9,875,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [4]:
# import team data and pre-process
filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0)#, engine='python')

# change team names to match convention used in the FPL data
teamStats.loc[teamStats['Squad']=='Brighton & Hove Albion','Squad'] = 'Brighton'
teamStats.loc[teamStats['Squad']=='Manchester United','Squad'] = 'Manchester Utd'
teamStats.loc[teamStats['Squad']=='Newcastle United','Squad'] = 'Newcastle Utd'
teamStats.loc[teamStats['Squad']=='Sheffield United','Squad'] = 'Sheffield Utd'
teamStats.loc[teamStats['Squad']=='West Ham United','Squad'] = 'West Ham'
teamStats.loc[teamStats['Squad']=='Tottenham Hotspur','Squad'] = 'Tottenham'
teamStats.loc[teamStats['Squad']=='Wolverhampton Wanderers','Squad'] = 'Wolves'

teamStats.head()

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Rk,,,,,,,,,,,,,,,,,,
1,Liverpool,35,30,3,2,76,27,49,93,66.0,37.6,28.5,0.81,W L W W D,53143,Mohamed Salah - 19,Alisson,NaN
2,Manchester City,35,23,3,9,91,34,57,72,85.8,32.0,53.7,1.53,L W L W W,54219,Raheem Sterling - 17,Ederson,NaN
3,Chelsea,35,18,6,11,63,49,14,60,62.1,36.4,25.6,0.73,W L W W L,40563,Tammy Abraham - 14,Kepa Arrizabalaga,NaN
4,Leicester City,35,17,8,10,65,36,29,59,55.7,42.8,13.0,0.37,D L W D L,32061,Jamie Vardy - 23,Kasper Schmeichel,NaN
5,Manchester Utd,35,16,11,8,61,35,26,59,55.3,33.6,21.7,0.62,W W W W D,72726,Marcus Rashford Anthony Martial - 16,David de Gea,NaN


In [5]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)

# fetch data locally
#df = pd.read_csv('../data/data_week' + str(latest_gameweek) + '.csv', index_col=0)#,encoding='latin-1')

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_saved,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,2,242,100.0,100.0,4,69140,0,0,-4,4,45.5,341,117,0,2,1.9,2.4,1,Shkodran,2.4,17,0,47.9,298,100,False,277.2,244,97,1205,,2020-07-01T22:00:25.971699Z,51,0,0,0,69140.jpg,2.9,0,0,Mustafi,0.4,False,None,a,1,3,155.0,225,54,43,23711,364,44908,503,0.5,8.4,Mustafi,2
2,0,4,204,100.0,100.0,4,98745,0,0,-2,2,76.9,293,89,0,2,2.2,2.7,1,Héctor,2.7,18,1,37.0,330,118,False,187.8,305,120,1156,,2019-12-09T20:00:21.228098Z,53,0,0,0,98745.jpg,2.9,0,0,Bellerín,1.0,False,None,a,1,3,103.0,266,75,44,198453,1087,150756,3334,0.5,8.3,Bellerín,2
3,2,1,310,100.0,100.0,4,111457,0,0,-3,3,167.3,210,51,0,2,1.9,2.4,1,Sead,2.4,24,0,48.7,293,98,False,249.6,262,105,1566,,2020-02-23T18:30:13.672943Z,52,0,0,0,111457.jpg,2.3,0,0,Kolasinac,0.5,False,None,a,1,3,73.0,307,100,52,62775,196,133247,330,0.5,10.0,Kolasinac,3
4,2,3,226,100.0,100.0,3,154043,0,0,-5,5,159.5,220,54,1,2,-0.1,0.4,0,Ainsley,0.4,20,0,47.0,305,102,False,279.2,242,96,1278,,2019-09-22T18:00:10.824841Z,45,0,0,0,154043.jpg,2.2,1,0,Maitland-Niles,2.2,False,None,a,1,3,37.0,357,130,39,608678,188,651180,888,0.1,8.7,Maitland-Niles,4
5,0,5,305,100.0,100.0,4,39476,0,0,-2,2,36.8,358,126,1,2,-0.5,0.0,0,Sokratis,0.0,25,2,58.5,253,79,False,436.2,150,54,1696,,2020-06-18T18:00:15.974146Z,48,0,0,0,39476.jpg,3.0,0,0,Papastathopoulos,1.3,False,None,a,1,3,110.0,260,73,57,181122,149,229381,808,0.0,11.9,Sokratis,6


## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [6]:
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xGA per game,probability no goals allowed
Rk,,,,,,,,,,,,,,,,,,,,
1,Liverpool,35,30,3,2,76,27,49,93,66.0,37.6,28.5,0.81,W L W W D,53143,Mohamed Salah - 19,Alisson,NaN,1.074286,0.341542
2,Manchester City,35,23,3,9,91,34,57,72,85.8,32.0,53.7,1.53,L W L W W,54219,Raheem Sterling - 17,Ederson,NaN,0.914286,0.400803
3,Chelsea,35,18,6,11,63,49,14,60,62.1,36.4,25.6,0.73,W L W W L,40563,Tammy Abraham - 14,Kepa Arrizabalaga,NaN,1.040000,0.353455
4,Leicester City,35,17,8,10,65,36,29,59,55.7,42.8,13.0,0.37,D L W D L,32061,Jamie Vardy - 23,Kasper Schmeichel,NaN,1.222857,0.294388
5,Manchester Utd,35,16,11,8,61,35,26,59,55.3,33.6,21.7,0.62,W W W W D,72726,Marcus Rashford Anthony Martial - 16,David de Gea,NaN,0.960000,0.382893
6,Wolves,35,14,13,8,48,37,11,55,45.0,31.0,14.0,0.40,W W L L W,31360,Raúl Jiménez - 16,Rui Patrício,NaN,0.885714,0.412419
7,Sheffield Utd,35,14,12,9,38,33,5,54,40.4,43.3,-2.9,-0.08,L W D W W,30869,Oliver McBurnie Lys Mousset - 6,Dean Henderson,NaN,1.237143,0.290212
8,Tottenham,35,14,10,11,54,45,9,52,44.3,49.3,-5.0,-0.14,W L W D W,59384,Harry Kane - 13,Paulo Gazzaniga,NaN,1.408571,0.244492
9,Arsenal,35,12,14,9,51,44,7,50,46.0,50.2,-4.2,-0.12,W W W D L,60279,Pierre-Emerick Aubameyang - 20,Bernd Leno,NaN,1.434286,0.238286


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

In [7]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, 'xG'].values[0]  
    penalty_attempts = playerStats.loc[indicator,'PKatt'].values[0]
    if df.loc[ix, 'element_type']<=2:
        # 0.24 is the probability to miss a penalty, which incurs -2 points
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] - 6 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  5 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts
    elif df.loc[ix, 'element_type']==4:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  4 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts

In [8]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[teamStats['Squad']==team, 'probability no goals allowed'].values[0]
    if df.loc[ix, 'element_type']<=2:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    4 * (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players assists based on their xA.

In [9]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, 'xA'].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)

In [10]:
def calculateAdjustedPoints(indicator, index):
    incorporate_xG(indicator, index)
    team_xGA(indicator, index)
    xA(indicator, index)

In [11]:
team_names = np.sort(playerStats['Squad'].unique())
xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
df['points_per_game'] = df['points_per_game'].astype(float)
df['games played'] = df['total_points']/df['points_per_game']
#df['games played'] = df['minutes']/90.0
xA_proportion

0.7649758454106279

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [12]:
for ix in df[df['minutes']>0].index: 
    # player name in FPL data
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    # find FBREF data indexes where player name contains 'name'
    indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains(name)
    # if unique match is found, we can calculate 'adjusted points'
    if playerStats.loc[indicator].shape[0]==1:
        calculateAdjustedPoints(indicator, ix)
    # if no match is found
    elif playerStats.loc[indicator].shape[0]==0:
        # try matching based on first name, helps in some cases
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[first_name_indicator].shape[0]==1:
            calculateAdjustedPoints(first_name_indicator, ix)
        # here we deal with some special cases individually
        else:
            names = ['rodrigo','garcia','chicharito','elliott']
            if name == 'rodrigo':
                exceptional_case_indicator = \
                (playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodri')) & \
                (~playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodriguez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'chicharito':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('javierhernandez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'elliott':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('elliot'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            else:
                print(str(ix) + ': no player found.')
    # if more than one matches were found in the original comparison
    elif playerStats.loc[indicator].shape[0]>1:
        # try matching full name
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[full_name_indicator].shape[0]==1:
            calculateAdjustedPoints(full_name_indicator, ix)
        # here we try to match players based both on their name and team
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats['Squad']==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                calculateAdjustedPoints(indicator & team_indicator, ix)
            # special case
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('heung')
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

In [13]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

In [14]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
191,Salah,30.821918,225,7.3,212.132620,6.882525
618,Fernandes,11.000000,99,9.0,72.807329,6.618848
281,McGovern,2.000000,11,5.5,12.661985,6.330993
215,De Bruyne,33.235294,226,6.8,206.767321,6.221318
214,Sterling,30.169492,178,5.9,177.137379,5.871408
192,Mané,32.063492,202,6.3,182.596669,5.694847
233,Rashford,27.931034,162,5.8,157.595507,5.642308
166,Vardy,32.063492,202,6.3,178.430435,5.564910
182,Alexander-Arnold,35.000000,196,5.6,181.186842,5.176767


In [15]:
# save data
filepath = '../data/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [16]:
total_xG = playerStats['xG'].sum()
total_goals = df['goals_scored'].sum()
print('Total xG: ' + str(total_xG))
print('Total goals: ' + str(total_goals))
print('Total xG / total goals: ' + str(total_xG/total_goals))

Total xG: 952.8999999999999
Total goals: 920
Total xG / total goals: 1.0357608695652172
